##### STEP 1 :Multi head training approach ,one head for reconstruction and another segmentation , backprop the two different IOU'S through the same encoder and decoder.

##### STEP 2: During inference recraete the segmented PC output with the usual feedforward through the net. Adapt the MISE algorithm with 2 values such that if the occupancy value is >0.5; then label that point with its class label too else discard that point. For this look in to the MISE code in occupancy nets.

###### STEP 3:NO NEED TO TRAIN AND MAKE DATASSET ,WE HAVE TRAINED OCCUPANCY WWE HAVE TRAINED SEGMENTATION NETWORK JUST PASS THE INPUT PC AND FIND OUT OCCUPANCY AND BEFORE ENDING FIND CLASS LABEL AT DESIRED RESOLUTION

##### Step 4: Predict label of each face/vertex


In [ ]:
!wget -c https://shapenet.cs.stanford.edu/ericyi/shapenetcore_partanno_segmentation_benchmark_v0.zip --no-check-certificate
!unzip shapenetcore_partanno_segmentation_benchmark_v0.zip
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
!pip install trimesh
#!unzip src.zip

In [ ]:
import os 
import torch
import h5py
import time
import json
from collections import defaultdict
import numpy as np
import torch.utils as utils
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
##seg
from seg_encoder import local_pool_pn
from seg_ref_encoder import LocalPoolPointnet
from seg_decoder import local_decoder
from seg_ref_decoder import LocalDecoder
from seg_trainer import Trainer
from seg_trainer import conv_seg_net
##occ
# from occ_encoder import local_pool_pn
# from occ_ref_encoder import LocalPoolPointnet
# from occ_decoder import local_decoder
# from occ_ref_decoder import LocalDecoder

In [ ]:
from seg_data import segment_Dataset
datapath="./shapenetcore_partanno_segmentation_benchmark_v0"
segmentation_dataset_train=segment_Dataset(root_path=datapath,class_choice=["Pistol"],split="train",data_augmentation=True)
segmentation_dataset_val=segment_Dataset(root_path=datapath,class_choice=["Pistol"],split="test",data_augmentation=True)

In [ ]:
print(segmentation_dataset_train[10][0].shape)
print(len(segmentation_dataset_train))

torch.Size([2500, 3])
209


In [ ]:
train_loader=utils.data.DataLoader(segmentation_dataset_train,batch_size=32,num_workers=4,shuffle=True)
val_loader=utils.data.DataLoader(segmentation_dataset_val,batch_size=32,num_workers=4,shuffle=True)

In [ ]:
seg_ref_encoder=LocalPoolPointnet(c_dim=32, dim=3, hidden_dim=32, scatter_type='max',unet=False, 
unet_kwargs=None, unet3d=True, unet3d_kwargs={"num_levels":3,"f_maps":32,"in_channels":32,
"out_channels":32}, plane_resolution=None,grid_resolution=32, plane_type='grid', padding=0.1, 
n_blocks=4)
seg_ref_decoder=LocalDecoder(dim=3, c_dim=32,hidden_size=32, n_blocks=4, leaky=False, 
                         sample_mode='bilinear', padding=0.1)

In [ ]:
seg_my_encoder=local_pool_pn(c_dim=32,mid_dim=32,scatter_type="max",unet3d=True,unet3d_kwargs={"num_levels":3,"f_maps":32,"in_channels":32,"out_channels":32},
                         grid_resolution=32,padding=0.1,n_blocks=4)

# ref_decoder=LocalDecoder(dim=3, c_dim=32,hidden_size=32, n_blocks=5, leaky=False, sample_mode='bilinear', padding=0.1)
seg_my_decoder=local_decoder(point_dim=3,feat_dim=32,leaky=False,sample_mode="bilinear",
                         padding=0.1,mid_dim=32,n_blocks=4)

In [ ]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
my_model=conv_seg_net(encoder=seg_my_encoder,decoder=seg_my_decoder,device=device)
my_model=my_model.to(device)
opt=optim.Adam(my_model.parameters(),lr=0.0001)
scheduler=optim.lr_scheduler.StepLR(opt,step_size=10,gamma=0.5)

In [ ]:
trainer=Trainer(model=my_model,optimizer=opt,device=device,vis_dir=None,threshold=0.5,eval_sample=False)

In [ ]:
next(seg_my_decoder.parameters()).device

device(type='cuda', index=0)

In [ ]:
my_model.decoder

local_decoder(
  (c_layer1): ModuleList(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=32, bias=True)
  )
  (p_layer1): Linear(in_features=3, out_features=32, bias=True)
  (res_blocks): ModuleList(
    (0): res_block(
      (fc1): Linear(in_features=32, out_features=32, bias=True)
      (fc2): Linear(in_features=32, out_features=32, bias=True)
    )
    (1): res_block(
      (fc1): Linear(in_features=32, out_features=32, bias=True)
      (fc2): Linear(in_features=32, out_features=32, bias=True)
    )
    (2): res_block(
      (fc1): Linear(in_features=32, out_features=32, bias=True)
      (fc2): Linear(in_features=32, out_features=32, bias=True)
    )
    (3): res_block(
      (fc1): Linear(in_features=32, out_features=32, bias=True)
      (fc2): Linear(in_features=32, out_features=32, bias=True)
    )
  )
  (

In [ ]:
n_epochs=500
for epoch in range(n_epochs):
  my_model.train()
  # if epoch % 10 == 0 and epoch > 0 :
  #   scheduler.step()
  for batch_idx,batch in enumerate(train_loader):
    
    points=batch[0].to(device)
    seg_op=batch[1].to(device)
    #encoder,decoder
    encoded_output=my_model.encoder(points)
    decoded_op=my_model.decoder(points,encoded_output)
    z=F.log_softmax(decoded_op,-1)
    re_dec1=z.view(-1,3)
    y1=seg_op.view(-1, 1)[:, 0] - 1
    loss=F.nll_loss(re_dec1,y1)
    loss.backward()
    opt.step()
    if batch_idx % 50 == 0:
      print("Training Loss for epoch %d == %.3f" % (epoch,loss))
      pred_choice=re_dec1.data.max(1)[1]
      correct=pred_choice.eq(y1.data).cpu().sum()
      print('[%d: %d] train loss: %f accuracy: %f' % (epoch,batch_idx, loss.item(), correct.item()/float(32 * 2500)))
    
    
  
  if epoch% 10 == 0:
    torch.save(my_model.state_dict(), 'seg_model_%s_%d.pth' % ("Rocket",epoch))
    ##val
    shape_ious=[]
    my_model.eval()
    for batch_idx,batch in enumerate(val_loader):
      
      points=batch[0].to(device)
      seg_op=batch[1].to(device)
      #encoder,decoder
      encoded_output=my_model.encoder(points)
      decoded_op=my_model.decoder(points,encoded_output)
      z=F.log_softmax(decoded_op,-1)
      pred_choice=z.data.max(2)[1]
      pred_np=pred_choice.cpu().data.numpy()
      target_np=seg_op.cpu().data.numpy() - 1
      

      for shape_idx in range(target_np.shape[0]):
        parts=range(4)
        part_ious=[]
        for part in parts:
          intersection=np.sum(np.logical_and(pred_np[shape_idx] == part, target_np[shape_idx] == part))
          union=np.sum(np.logical_or(pred_np[shape_idx] == part, target_np[shape_idx] == part))
          if union==0:
            iou=1
          else:
            iou=intersection/float(union)
          part_ious.append(iou)
        shape_ious.append(np.mean(part_ious))
    print("Mean IOU for shape on validation set:",np.mean(shape_ious))

Training Loss for epoch 0 == 1.194
[0: 0] train loss: 1.193819 accuracy: 0.301513
Mean IOU for shape on validation set: 0.47816684661651815
Training Loss for epoch 1 == 0.688
[1: 0] train loss: 0.687515 accuracy: 0.684538
Training Loss for epoch 2 == 0.623
[2: 0] train loss: 0.623452 accuracy: 0.726862
Training Loss for epoch 3 == 0.567
[3: 0] train loss: 0.566984 accuracy: 0.825187
Training Loss for epoch 4 == 0.488
[4: 0] train loss: 0.487711 accuracy: 0.831187
Training Loss for epoch 5 == 0.526
[5: 0] train loss: 0.526252 accuracy: 0.792675
Training Loss for epoch 6 == 0.498
[6: 0] train loss: 0.498236 accuracy: 0.869287
Training Loss for epoch 7 == 0.554
[7: 0] train loss: 0.554154 accuracy: 0.881262
Training Loss for epoch 8 == 0.428
[8: 0] train loss: 0.427757 accuracy: 0.852000
Training Loss for epoch 9 == 0.579
[9: 0] train loss: 0.579292 accuracy: 0.749762
Training Loss for epoch 10 == 0.668
[10: 0] train loss: 0.667586 accuracy: 0.692750
Mean IOU for shape on validation set: 

KeyboardInterrupt: ignored

In [ ]:
import yaml
from get_dataset import get_data_fields
from get_dataset import Shapes3dDataset
from get_dataset import get_inputs_field
from get_dataset import collate_remove_none
!unzip small_chair.zip
def load_config(path, default_path=None):
    with open(path, 'r') as f:
        cfg_special = yaml.load(f)
    inherit_from = cfg_special.get('inherit_from')
    if inherit_from is not None:
        cfg = load_config(inherit_from, default_path)
    elif default_path is not None:
        with open(default_path, 'r') as f:
            cfg = yaml.load(f)
    else:
        cfg = dict()
    update_recursive(cfg, cfg_special)
    return cfg


def update_recursive(dict1, dict2):
    for k, v in dict2.items():
        if k not in dict1:
            dict1[k] = dict()
        if isinstance(v, dict):
            update_recursive(dict1[k], v)
        else:
            dict1[k] = v

In [ ]:
cfg1=load_config("./shapenet_grid32.yaml","./default.yaml")
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from src import data
cfg=cfg1
mode="val"
method = cfg['method']
dataset_type = cfg['data']['dataset']
dataset_folder = cfg['data']['path']
categories = cfg['data']['classes']
print(dataset_folder)

splits = {'train': cfg['data']['train_split'],'val': cfg['data']['val_split'],'test': cfg['data']['test_split'],}
split = splits[mode]
return_idx=True
if dataset_type == 'Shapes3D':
    fields=get_data_fields(mode,cfg1)
    inputs_field = get_inputs_field(mode, cfg1)
    if inputs_field is not None:
        fields['inputs'] = inputs_field
    if return_idx:
        fields['idx'] = data.IndexField()

val_dataset=Shapes3dDataset(dataset_folder, fields,split=split,categories=categories,cfg=cfg1)

./small_chair


In [ ]:
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=1, num_workers=cfg['training']['n_workers_val'], shuffle=False,
    collate_fn=collate_remove_none)
seg_val_loader=utils.data.DataLoader(segmentation_dataset_val,batch_size=1,num_workers=4,shuffle=False)
for batch_idx, batch in enumerate(val_loader):
  if batch_idx == 0:
    print(batch.keys())
    p=batch["inputs"]
    print(batch["inputs"].shape)
  else:
    break
for batch_idx, batch in enumerate(seg_val_loader):
  if batch_idx == 0:
    t=batch[0]
    q=batch[1]
    print(batch[0].shape)
  else:
    break

dict_keys(['points', 'points.occ', 'points_iou', 'points_iou.occ', 'inputs', 'inputs.normals', 'idx'])
torch.Size([1, 3000, 3])
torch.Size([1, 3000, 3])


In [ ]:
p

tensor([[[ 0.0764,  0.0752,  0.0381],
         [-0.2227,  0.3505,  0.0338],
         [ 0.0402,  0.0174,  0.0961],
         ...,
         [-0.1530,  0.0033, -0.2361],
         [-0.2116, -0.1579, -0.1993],
         [-0.1396,  0.0169, -0.0365]]])

In [ ]:
state_dict=torch.load("trained_seg_model.pth")
seg_my_model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
#cerate occ model and load all weights
occ_ref_encoder=LocalPoolPointnet(c_dim=32, dim=3, hidden_dim=32, scatter_type='max',unet=False, 
unet_kwargs=None, unet3d=True, unet3d_kwargs={"num_levels":3,"f_maps":32,"in_channels":32,
"out_channels":32}, plane_resolution=None,grid_resolution=32, plane_type='grid', padding=0.1, 
n_blocks=5)
occ_ref_decoder=LocalDecoder(dim=3, c_dim=32,hidden_size=32, n_blocks=5, leaky=False, 
                         sample_mode='bilinear', padding=0.1)

occ_my_model=conv_seg_net(encoder=occ_ref_encoder,decoder=occ_ref_decoder,device=device)
occ_my_model=occ_my_model.to(device)

In [ ]:
occ_state_dict=torch.load("shapenet_grid32.pt")
occ_my_model.load_state_dict(occ_state_dict["model"])

<All keys matched successfully>

In [ ]:
!python setup.py build_ext --inplace 

running build_ext
/usr/local/lib/python3.6/dist-packages/torch/utils/cpp_extension.py:339: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
copying build/lib.linux-x86_64-3.6/src/utils/libkdtree/pykdtree/kdtree.cpython-36m-x86_64-linux-gnu.so -> src/utils/libkdtree/pykdtree
copying build/lib.linux-x86_64-3.6/src/utils/libmcubes/mcubes.cpython-36m-x86_64-linux-gnu.so -> src/utils/libmcubes
copying build/lib.linux-x86_64-3.6/src/utils/libmesh/triangle_hash.cpython-36m-x86_64-linux-gnu.so -> src/utils/libmesh
copying build/lib.linux-x86_64-3.6/src/utils/libmise/mise.cpython-36m-x86_64-linux-gnu.so -> src/utils/libmise
copying build/lib.linux-x86_64-3.6/src/utils/libsimplify/simplify_mesh.cpython-36m-x86_64-linux-gnu.so -> src/utils/libsimplify
copying build/lib.linux-x86_64-3.6/src/utils/libvoxelize/voxelize.cpython-36m-x86_64-linux-gnu.s

In [ ]:
from src.utils.libmise import MISE
from src.utils import libmcubes

import trimesh

def add_key(base, new, base_name, new_name, device=None):
    ''' Add new keys to the given input

    Args:
        base (tensor): inputs
        new (tensor): new info for the inputs
        base_name (str): name for the input
        new_name (str): name for the new info
        device (device): pytorch device
    '''
    if (new is not None) and (isinstance(new, dict)):
        if device is not None:
            for key in new.keys():
                new[key] = new[key].to(device)
        base = {base_name: base,
                new_name: new}
    return base

def make_3d_grid(bb_min, bb_max, shape):
    ''' Makes a 3D grid.

    Args:
        bb_min (tuple): bounding box minimum
        bb_max (tuple): bounding box maximum
        shape (tuple): output shape
    '''
    size = shape[0] * shape[1] * shape[2]

    pxs = torch.linspace(bb_min[0], bb_max[0], shape[0])
    pys = torch.linspace(bb_min[1], bb_max[1], shape[1])
    pzs = torch.linspace(bb_min[2], bb_max[2], shape[2])

    pxs = pxs.view(-1, 1, 1).expand(*shape).contiguous().view(size)
    pys = pys.view(1, -1, 1).expand(*shape).contiguous().view(size)
    pzs = pzs.view(1, 1, -1).expand(*shape).contiguous().view(size)
    p = torch.stack([pxs, pys, pzs], dim=1)

    return p

In [ ]:
vis_loader = torch.utils.data.DataLoader(
    segmentation_dataset_val, batch_size=1, shuffle=False)
model_counter = defaultdict(int)
data_vis_list = []

In [ ]:
iterator=iter(vis_loader)
for i in range(len(vis_loader)):
    data_vis = next(iterator)
    data_vis_list.append({"category":"Airplane","data":data_vis})

In [ ]:
print(len(data_vis_list))

341


In [ ]:
sample=data_vis_list[0]["data"][0]
print(sample.shape)
sample=sample.to(device)
stats_dict = {}
occ_my_model.eval()
with torch.no_grad():
  c=occ_my_model.encoder(sample)
#print(c["grid"].shape)
th=0.5
threshold=np.log(th)-np.log(1.-th)
box_size=1 + 0.1
mesh_extractor=MISE(32,2,threshold)#since upsampling is done twice 32==>64==>128
points=mesh_extractor.query()##################### (32+1)*(32+1)*(32+1)
print("##",points.shape)
#print("##",points)
#while points.shape[0]  != 0:
while points.shape[0] != 0 :
  pf=points/mesh_extractor.resolution
  #print("#",pf.shape)
  #print("#",pf)
  pointsf = box_size * (pf - 0.5)
  pointsf=torch.FloatTensor(pointsf).to(device)


  p_split=torch.split(pointsf,100000)
  #print(p_split[0].shape)
  ##eval pointsf
  occ_hats=[]
  for pi in p_split:
    pi=pi.unsqueeze(0)
    #print(pi.shape)
    with torch.no_grad():
      occ_hat1=occ_my_model.decoder(pi,c)
      print("Attention:",occ_hat1.shape)
      p_r=torch.distributions.Bernoulli(logits=occ_hat1).logits

    occ_hats.append(p_r.squeeze(0).detach().cpu())
  occ_hat=torch.cat(occ_hats,dim=0)
  ## end eval points
  occ_hat=occ_hat.cpu().numpy()
  values=occ_hat.astype(np.float64)
  print("Points in mesh",points.shape)
  print("Values of points",values.shape)
  mesh_extractor.update(points,values)
  points=mesh_extractor.query()


value_grid = mesh_extractor.to_dense()
#print("Valeu grid",value_grid)
print("$$$$$",value_grid.shape)
##############################################################
n_x, n_y, n_z = value_grid.shape
# Make sure that mesh is watertight
t0 = time.time()
occ_hat_padded = np.pad(value_grid, 1, 'constant', constant_values=-1e6)
vertices, triangles = libmcubes.marching_cubes(value_grid, threshold)
print(vertices.shape)
print(triangles.shape)
# libmcubes traingles are the faces , we have the points , just check all points the faces contain and then label that mesh by majority voting 
#stats_dict['time (marching cubes)'] = time.time() - t0
# Strange behaviour in libmcubes: vertices are shifted by 0.5
vertices -= 0.5
# # Undo padding
vertices -= 1
vertices /= np.array([n_x-1, n_y-1, n_z-1])
vertices = box_size * (vertices - 0.5)
normals=None
mesh=trimesh.Trimesh(vertices, triangles,vertex_normals=normals,process=False)
#mesh.export("./x8.off")
#mesh.export(os.path.join(out_dir, 'vis', '{}_{}_{}.off'.format(it, data_vis['category'], data_vis['it'])))


torch.Size([1, 3000, 3])
## (35937, 3)
Attention: torch.Size([1, 35937])
Points in mesh (35937, 3)
Values of points (35937,)
Attention: torch.Size([1, 5604])
Points in mesh (5604, 3)
Values of points (5604,)
Attention: torch.Size([1, 22126])
Points in mesh (22126, 3)
Values of points (22126,)
Attention: torch.Size([1, 1812])
Points in mesh (1812, 3)
Values of points (1812,)
Attention: torch.Size([1, 668])
Points in mesh (668, 3)
Values of points (668,)
Attention: torch.Size([1, 413])
Points in mesh (413, 3)
Values of points (413,)
Attention: torch.Size([1, 109])
Points in mesh (109, 3)
Values of points (109,)
Attention: torch.Size([1, 24])
Points in mesh (24, 3)
Values of points (24,)
$$$$$ (129, 129, 129)
(10154, 3)
(19596, 3)


In [ ]:
mesh.export("./x.off")

'OFF\n10154 19596 0\n-0.5549805104 0.0429687500 -0.2234375000\n-0.5500000000 0.0406675506 -0.2234375000\n-0.5500000000 0.0429687500 -0.2273517412\n-0.5500000000 0.0375908060 -0.2148437500\n-0.5585937500 0.0411648749 -0.2148437500\n-0.5585937500 0.0429687500 -0.2177732625\n-0.5500000000 0.0359088736 -0.2062500000\n-0.5585937500 0.0411648749 -0.2148437500\n-0.5585937500 0.0371074463 -0.2062500000\n-0.5500000000 0.0357117966 -0.1976562500\n-0.5585937500 0.0371074463 -0.2062500000\n-0.5585937500 0.0363592679 -0.1976562500\n-0.5500000000 0.0361177700 -0.1890625000\n-0.5585937500 0.0363592679 -0.1976562500\n-0.5585937500 0.0364216653 -0.1890625000\n-0.5500000000 0.0364741025 -0.1804687500\n-0.5585937500 0.0364216653 -0.1890625000\n-0.5585937500 0.0366580184 -0.1804687500\n-0.5500000000 0.0366355381 -0.1718750000\n-0.5585937500 0.0366580184 -0.1804687500\n-0.5585937500 0.0370632442 -0.1718750000\n-0.5500000000 0.0366464522 -0.1632812500\n-0.5585937500 0.0370632442 -0.1718750000\n-0.5585937500

##### we want to sample log odds so for that we either use F.softmax==>sample from it==>then log(p/1-p).
##### OR use F.log_softmax==>
##### The p can be 2 things,1: the max sampled value or , that will be the probability apply log odds for 1 would be 

##### labels for faces OR vertices OR edges IF it makes a mesh through MISE. look into this line:
##### vertices, triangles = libmcubes.marching_cubes(value_grid, threshold)
  